In [1]:
!pip install pandas matplotlib numpy

In [2]:
!pip install bar_chart_race

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 2.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
import bar_chart_race as bcr

# Extrutura inicial do DataSet
### Abrange ano de 2014 a 2025

|NOME|DESCRIÇÃO|TIPO|
|----|---------|----|
|ANO|Ano civil, 1o de Janeiro a 31 de Dezembro. |Number(4)|
|NOME_SALA|Nome da sala relacionada ao seu cadastro na ANCINE. |String(150)|
|REGISTRO_SALA | Código de identificação do registro da sala na ANCINE.|String(7)|
|MUNICIPIO        | Município referente ao endereço da sala.                    | String(50)|
|UF               | Sigla da unidade federativa referente ao endereço da sala.  | String(2)|
|STATUS           | 'ABERTA','FECHADA','FECHADA TEMPORARIAMENTE'                | String(23)|




In [4]:
url = "https://dados.ancine.gov.br/dados-abertos/salas-de-exibicao-evolucao-anual.json"
df = pd.read_json(url)
df = pd.json_normalize(df['data'])
# Adicionando a coluna PAIS
df['PAIS'] = "Brasil"
# Reordenando as colunas
nova_ordem = ['PAIS', 'UF','MUNICIPIO', 'ANO', 'STATUS', 'REGISTRO_ANCINE']
# Atribuindo o reordenamento ao DataFrame
df = df[nova_ordem]

In [5]:
print("Colunas disponíveis:")
print(df.columns.tolist())

Colunas disponíveis:
['PAIS', 'UF', 'MUNICIPIO', 'ANO', 'STATUS', 'REGISTRO_ANCINE']


In [6]:
# Dicionário de estados (como fornecido)
nome_estado = {
    "AC": "Acre",
    "AL": "Alagoas",
    "AP": "Amapá",
    "AM": "Amazonas",
    "BA": "Bahia",
    "CE": "Ceará",
    "DF": "Distrito Federal",
    "ES": "Espírito Santo",
    "GO": "Goiás",
    "MA": "Maranhão",
    "MT": "Mato Grosso",
    "MS": "Mato Grosso do Sul",
    "MG": "Minas Gerais",
    "PA": "Pará",
    "PB": "Paraíba",
    "PR": "Paraná",
    "PE": "Pernambuco",
    "PI": "Piauí",
    "RJ": "Rio de Janeiro",
    "RN": "Rio Grande do Norte",
    "RS": "Rio Grande do Sul",
    "RO": "Rondônia",
    "RR": "Roraima",
    "SC": "Santa Catarina",
    "SP": "São Paulo",
    "SE": "Sergipe",
    "TO": "Tocantins"
}

# Passo 1: Renomear a coluna 'UF' para 'Estado'
df = df.rename(columns={'UF': 'ESTADO'})

# Passo 2: Renomear a coluna 'REGISTRO_ANCINE' para 'ID'
df = df.rename(columns={'REGISTRO_ANCINE': 'ID'})

# Passo 3: Mapear os valores da coluna 'Estado' usando o dicionário
df['ESTADO'] = df['ESTADO'].map(nome_estado)

# Verificar o resultado
print(df.head())

     PAIS       ESTADO MUNICIPIO   ANO   STATUS       ID
0  Brasil  Mato Grosso    CUIABÁ  2014  FECHADA  5002041
1  Brasil  Mato Grosso    CUIABÁ  2014  FECHADA  5002040
2  Brasil  Mato Grosso    CUIABÁ  2014  FECHADA  5002042
3  Brasil  Mato Grosso    CUIABÁ  2014  FECHADA  5002043
4  Brasil  Mato Grosso    CUIABÁ  2014  FECHADA  5002038


# Extrutura atual do DataSet
### Abrange ano de 2014 a 2025

|NOME|DESCRIÇÃO|TIPO|
|----|---------|----|
|PAIS|Nome do País|String(6)|
|ESTADO|Nome do Estado|String(2)|
|MUNICIPIO|Nome do Município| String(50)|
|ANO|Ano civil|Number(4)|
|STATUS| 'ABERTA','FECHADA','FECHADA TEMPORARIAMENTE'| String(23)|
|ID | Código de identificação do registro da sala na ANCINE.|String(7)|


In [7]:
# Verificar o resultado
print(df.head())

     PAIS       ESTADO MUNICIPIO   ANO   STATUS       ID
0  Brasil  Mato Grosso    CUIABÁ  2014  FECHADA  5002041
1  Brasil  Mato Grosso    CUIABÁ  2014  FECHADA  5002040
2  Brasil  Mato Grosso    CUIABÁ  2014  FECHADA  5002042
3  Brasil  Mato Grosso    CUIABÁ  2014  FECHADA  5002043
4  Brasil  Mato Grosso    CUIABÁ  2014  FECHADA  5002038


In [8]:
"""
    Na coluna STATUS vamos colocar
    1 onde estiver "ABERTA" e
    0 onde estiver "FECHADA" ou "FECHADA TEMPORARIAMENTE"

    Com isso podemos somar e agrupar para saber o total de salas abertas.
    Por exemplo:
      Total salas abertas em Mato Grosso em 2014.
      Total salas abertas em Mato Grosso em 2015.
      Total salas abertas em Mato Grosso em 2016.
"""
df['STATUS'] = df['STATUS'].apply(
    lambda x: 1 if x == 'ABERTA' else 0
)
# Verificar o resultado
print(df.head())
df_modelo = df

     PAIS       ESTADO MUNICIPIO   ANO  STATUS       ID
0  Brasil  Mato Grosso    CUIABÁ  2014       0  5002041
1  Brasil  Mato Grosso    CUIABÁ  2014       0  5002040
2  Brasil  Mato Grosso    CUIABÁ  2014       0  5002042
3  Brasil  Mato Grosso    CUIABÁ  2014       0  5002043
4  Brasil  Mato Grosso    CUIABÁ  2014       0  5002038


# Data Frame dos Estados

In [9]:
# Limitando as colunas
nova_ordem = ['ESTADO', 'ANO', 'STATUS', 'ID']
# Atribuindo o reordenamento ao DataFrame
df_estado = df_modelo[nova_ordem]
# Agrupar por estado e ANO, somando STATUS
df_estado = df_estado.groupby(['ESTADO', 'ANO'])['STATUS'].sum().reset_index()
# Renomear a coluna 'STATUS' para 'TOTAL' que representa o TOTAL DE SALAS ABERTAS
df_estado = df_estado.rename(columns={'STATUS': 'TOTAL'})
# Verificar o resultado
print(df_estado.head())

  ESTADO   ANO  TOTAL
0   Acre  2014      5
1   Acre  2015      5
2   Acre  2016      5
3   Acre  2017      7
4   Acre  2018      7


In [10]:
# Renomeie a coluna 'ANO' para 'DATA' (opcional, só se quiser trocar o nome)
df_estado = df_estado.rename(columns={'ANO': 'DATA'})

# Ordene o DataFrame pela data (importante para o bar_chart_race)
df_estado = df_estado.sort_values('DATA')

# Converta a coluna 'DATA' para datetime (usando a coluna renomeada)
df_estado['DATA'] = pd.to_datetime(df_estado['DATA'].astype(str), format='%Y')

# Pivotear o DataFrame (formato exigido pelo bar_chart_race)
df_pivot = df_estado.pivot(index='DATA', columns='ESTADO', values='TOTAL').fillna(0)

# DF_ESTADO

In [11]:
df_estado.head()

,ESTADO,DATA,TOTAL
0,Acre,2014-01-01,5
84,Espírito Santo,2014-01-01,60
228,Rio Grande do Sul,2014-01-01,157
48,Bahia,2014-01-01,78
216,Rio Grande do Norte,2014-01-01,31


# DF_PIVOT

In [12]:
df_pivot.head()

ESTADO,Acre,Alagoas,Amapá,Amazonas,Bahia,Ceará,Distrito Federal,Espírito Santo,Goiás,Maranhão,...,Piauí,Rio Grande do Norte,Rio Grande do Sul,Rio de Janeiro,Rondônia,Roraima,Santa Catarina,Sergipe,São Paulo,Tocantins
DATA,,,,,,,,,,,,,,,,,,,,,
2014-01-01,5,29,7,55,78,77,81,60,96,33,...,12,31,157,333,15,6,104,19,911,14
2015-01-01,5,29,17,63,84,85,86,69,99,44,...,24,31,145,341,15,14,120,19,970,14
2016-01-01,5,29,17,63,96,98,87,72,111,51,...,23,31,157,357,15,14,129,21,984,17
2017-01-01,7,29,17,71,104,100,88,73,117,60,...,26,31,176,356,18,15,131,25,1031,18
2018-01-01,7,29,17,64,109,100,88,73,112,60,...,30,31,177,368,19,15,148,25,1026,18


# Extrutura atual do DataSet
### Abrange ano de 2014 a 2025

|NOME|DESCRIÇÃO|TIPO|
|----|---------|----|
|ESTADO|Nome do Estado|String(2)|
|ANO|Ano civil|Number(4)|
|TOTAL | Total de salas de cinema da ANCINE abertas no Estado|String(7)|

In [13]:
bcr.bar_chart_race(
    df=df_pivot,
    filename='salas_cinema_por_estado_brasil.mp4',
    orientation='h',
    title='Salas de cinema da ANCINE ativas por Estado (2014-2025)',
    title_size=14,
    bar_label_size=8,
    tick_label_size=7
)

/usr/local/lib/python3.11/dist-packages/bar_chart_race/_make_chart.py:889: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
/usr/local/lib/python3.11/dist-packages/bar_chart_race/_make_chart.py:286: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(self.df_values.columns)
/usr/local/lib/python3.11/dist-packages/bar_chart_race/_make_chart.py:287: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))


In [14]:
mapeamento_regioes = {
    # Norte
    'Amazonas': 'NORTE',
    'Roraima': 'NORTE',
    'Amapá': 'NORTE',
    'Pará': 'NORTE',
    'Tocantins': 'NORTE',
    'Rondônia': 'NORTE',
    'Acre': 'NORTE',
    # Nordeste
    'Maranhão': 'NORDESTE',
    'Piauí': 'NORDESTE',
    'Ceará': 'NORDESTE',
    'Rio Grande do Norte': 'NORDESTE',
    'Pernambuco': 'NORDESTE',
    'Paraíba': 'NORDESTE',
    'Sergipe': 'NORDESTE',
    'Alagoas': 'NORDESTE',
    'Bahia': 'NORDESTE',
    # Centro-Oeste
    'Mato Grosso': 'CENTRO-OESTE',
    'Mato Grosso do Sul': 'CENTRO-OESTE',
    'Goiás': 'CENTRO-OESTE',
    'Distrito Federal': 'CENTRO-OESTE',
    # Sudeste
    'São Paulo': 'SUDESTE',
    'Rio de Janeiro': 'SUDESTE',
    'Espírito Santo': 'SUDESTE',
    'Minas Gerais': 'SUDESTE',
    # Sul
    'Paraná': 'SUL',
    'Rio Grande do Sul': 'SUL',
    'Santa Catarina': 'SUL'
}
df_regiao = df_estado
df_regiao['REGIAO'] = df_estado['ESTADO'].map(mapeamento_regioes)
# Limitando as colunas
nova_ordem = ['REGIAO','ESTADO', 'DATA', 'TOTAL']
# Atribuindo o reordenamento ao DataFrame
df_regiao = df_regiao[nova_ordem]
df_regiao.head()

,REGIAO,ESTADO,DATA,TOTAL
0,NORTE,Acre,2014-01-01,5
84,SUDESTE,Espírito Santo,2014-01-01,60
228,SUL,Rio Grande do Sul,2014-01-01,157
48,NORDESTE,Bahia,2014-01-01,78
216,NORDESTE,Rio Grande do Norte,2014-01-01,31


In [15]:
# Limitando as colunas
nova_ordem = ['REGIAO', 'DATA', 'TOTAL']
# Atribuindo o reordenamento ao DataFrame
df_regiao = df_regiao[nova_ordem]
# Ordene o DataFrame pela data (importante para o bar_chart_race)
df_regiao = df_regiao.sort_values('DATA')
# Agrupar e somar
df_regiao = df_regiao.groupby(['REGIAO', 'DATA'])['TOTAL'].sum().reset_index()
# Pivotear o DataFrame (formato exigido pelo bar_chart_race)
df_pivot = df_regiao.pivot(index='DATA', columns='REGIAO', values='TOTAL').fillna(0)
df_pivot.head()

REGIAO,CENTRO-OESTE,NORDESTE,NORTE,SUDESTE,SUL
DATA,,,,,
2014-01-01,239,394,152,1536,434
2015-01-01,255,437,191,1627,428
2016-01-01,270,481,196,1668,463
2017-01-01,278,509,211,1716,493
2018-01-01,280,539,223,1735,523


In [16]:
bcr.bar_chart_race(
    df=df_pivot,
    filename='salas_cinema_por_regiao_brasil.mp4',
    orientation='h',
    title='Salas de cinema da ANCINE ativas por Região (2014-2025)',
    title_size=14,
    bar_label_size=8,
    tick_label_size=7
)

/usr/local/lib/python3.11/dist-packages/bar_chart_race/_make_chart.py:889: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
/usr/local/lib/python3.11/dist-packages/bar_chart_race/_make_chart.py:286: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(self.df_values.columns)
/usr/local/lib/python3.11/dist-packages/bar_chart_race/_make_chart.py:287: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
